In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2024-08-10 18:15:39.670227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 18:15:39.670375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 18:15:39.846437: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv


In [2]:
# Load the dataset
train_data = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test_data = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# Combine prompts and responses for feature extraction
train_data['text_a'] = train_data['prompt'] + ' ' + train_data['response_a']
train_data['text_b'] = train_data['prompt'] + ' ' + train_data['response_b']

# Vectorize the text data using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_a = tfidf.fit_transform(train_data['text_a']).toarray()
X_b = tfidf.transform(train_data['text_b']).toarray()

# Combine the vectorized responses
X = np.hstack([X_a, X_b])

# Target variable
y = train_data[['winner_model_a', 'winner_model_b', 'winner_tie']].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [4]:
# we'll learn the values here
# Define dropout rates and activation functions to test
dropout_rates = [0.3, 0.5, 0.7]
activation_functions = ['relu', 'tanh', 'sigmoid']
density_vals = [600, 700, 800]

# Store results
results = []

for density in density_vals:
    for dropout in dropout_rates:
        for activation in activation_functions:
            model = Sequential([
                Dense(density, activation=activation, input_shape=(X_train.shape[1],)),
                Dropout(dropout),
                Dense(density//2, activation=activation),
                Dropout(dropout),
                Dense(density//4, activation=activation),
                Dropout(dropout),
                Dense(3, activation='softmax')
            ])
            
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=0)
            val_predictions = model.predict(X_val)
            loss = log_loss(y_val, val_predictions)
            
            results.append((dropout, activation, density, loss))

# Find the best configuration
best_config = min(results, key=lambda x: x[2])

# feed best parameters
best_dropout, best_activation, best_density, _ = best_config

360/360 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


360/360 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


360/360 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


...

360/360 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


360/360 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


360/360 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step


In [5]:
# Build a simple neural network model
model = Sequential([
    Dense(best_density, activation=best_activation, input_shape=(X_train.shape[1],)),
    Dropout(best_dropout),
    Dense(best_density/2, activation=best_activation),
    Dropout(best_dropout),
    Dense(best_density/4, activation=best_activation),
    Dropout(best_dropout),
    Dense(3, activation=best_activation)
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
val_predictions = model.predict(X_val)
loss = log_loss(y_val, val_predictions)
print(f'Validation Log Loss: {loss}')

# Prepare the test data
test_data['text_a'] = test_data['prompt'] + ' ' + test_data['response_a']
test_data['text_b'] = test_data['prompt'] + ' ' + test_data['response_b']
X_test_a = tfidf.transform(test_data['text_a']).toarray()
X_test_b = tfidf.transform(test_data['text_b']).toarray()
X_test = np.hstack([X_test_a, X_test_b])
X_test = scaler.transform(X_test)

# Predict on the test set
test_predictions = model.predict(X_test)

# Create the submission file
submission = pd.DataFrame(test_data['id'])
submission['winner_model_a'] = test_predictions[:, 0]
submission['winner_model_b'] = test_predictions[:, 1]
submission['winner_tie'] = test_predictions[:, 2]
submission.to_csv('submission.csv', index=False)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 3), output.shape=(None, 85)